In [74]:
import pandas as pd
import random
import itertools

In [75]:
# Function to simulate a prisoner's dilemma game
def play_prisoners_dilemma(contestants, iterations=5):
    # Initialize a DataFrame to store results
    results = pd.DataFrame(columns=['Contestant1', 'Contestant2', 'Round', 'C1_Decision', 'C2_Decision', 'C1_Payoff', 'C2_Payoff'])

    # Define payoff matrix
    payoff_matrix = {
        ('Cooperate', 'Cooperate'): (2, 2),
        ('Cooperate', 'Defect'): (0, 3),
        ('Defect', 'Cooperate'): (3, 0),
        ('Defect', 'Defect'): (1, 1)
    }

    # Initialize a dictionary to store total scores
    total_scores = {contestant: 0 for contestant in contestants.keys()}

    # Initialize a dictionary to store matchup scores
    matchup_scores = {c1: {c2: 0 for c2 in contestants.keys()} for c1 in contestants.keys()}

    # Iterate over all pairs of contestants
    for c1, c2 in itertools.combinations(contestants.keys(), 2):
        # Initialize history DataFrames for both contestants
        c1_history = pd.DataFrame(columns=['Round', 'Opponent_Decision', 'My_Decision'])
        c2_history = pd.DataFrame(columns=['Round', 'Opponent_Decision', 'My_Decision'])

        for round_number in range(1, iterations + 1):
            # Get decisions from both contestants
            c1_decision = contestants[c1](c1_history)
            c2_decision = contestants[c2](c2_history)

            # Record decisions in results DataFrame
            c1_payoff, c2_payoff = payoff_matrix[(c1_decision, c2_decision)]
            new_result = pd.DataFrame([{
                'Contestant1': c1,
                'Contestant2': c2,
                'Round': round_number,
                'C1_Decision': c1_decision,
                'C2_Decision': c2_decision,
                'C1_Payoff': c1_payoff,
                'C2_Payoff': c2_payoff
            }])
            results = pd.concat([results, new_result], ignore_index=True)

            # Update total scores
            total_scores[c1] += c1_payoff
            total_scores[c2] += c2_payoff

            # Update matchup scores
            matchup_scores[c1][c2] += c1_payoff
            matchup_scores[c2][c1] += c2_payoff

            # Update history DataFrames
            new_c1_history = pd.DataFrame([{'Round': round_number, 'Opponent_Decision': c2_decision, 'My_Decision': c1_decision}])
            new_c2_history = pd.DataFrame([{'Round': round_number, 'Opponent_Decision': c1_decision, 'My_Decision': c2_decision}])
            c1_history = pd.concat([c1_history, new_c1_history], ignore_index=True)
            c2_history = pd.concat([c2_history, new_c2_history], ignore_index=True)

    return results, total_scores, matchup_scores



In [83]:
# Example contestant strategies
def tit_for_tat_nice(history):
    """Cooperates the first turn, then tit for tat thereafter
    """
    if not history.empty and history['Opponent_Decision'].iloc[-1] == 'Defect':
        return 'Defect'
    return 'Cooperate'

def tit_for_tat_forgiveness(history):
    """Even if the opponent defects, there is a 10% chance it will Cooperate the next turn anyway
    """
    if not history.empty and history['Opponent_Decision'].iloc[-1] == 'Defect' and random.random() < 0.9:
            return 'Defect'
    return 'Cooperate'

def gaslighter(history):
    """
    Cooperates the first three turns, then defects thereafter.
    """
    if history.empty or len(history) < 3:
        return 'Cooperate'
    return 'Defect'

def always_cooperate(history):
    return 'Cooperate'

def alternating_strategy(history):
    """
    Defects the first three times, then cooperates the next three times, and so on.
    """
    # Determine the current round
    current_round = len(history) + 1

    # Determine the phase of the strategy
    phase = (current_round - 1) // 5

    if phase % 2 == 0:
        return 'Defect'
    else:
        return 'Cooperate'

def random_choice(history):
    return random.choice(["Defect", "Cooperate"])

# Define contestants
contestants = {
    'Axel': tit_for_tat_nice,
    'Jono': tit_for_tat_forgiveness,
    'Cheng': gaslighter,
    'Gwyn': always_cooperate,
    'Jacob': alternating_strategy,
    'Touhid': random_choice
}

In [85]:
# Run the prisoner's dilemma simulation
results, total_scores, matchup_scores = play_prisoners_dilemma(contestants, iterations=50)

# Display results
print("Matchup Results:")
print(results)

# Display the score matrix with hyphens for self-intersections
matchup_scores_df = pd.DataFrame(matchup_scores).fillna(0).astype(int).astype(str)

# Substitute diagonal values with hyphens
for contestant in contestants.keys():
    matchup_scores_df.loc[contestant, contestant] = '-'

print("\nScore Matrix:")
print(matchup_scores_df.T)

# Rank the contestants by their total scores
total_scores_df = pd.DataFrame(list(total_scores.items()), columns=['Contestant', 'Total_Score'])
ranked_scores_df = total_scores_df.sort_values(by='Total_Score', ascending=False).reset_index(drop=True)
print("\nRanked Scores:")
print(ranked_scores_df)


Matchup Results:
    Contestant1 Contestant2 Round C1_Decision C2_Decision C1_Payoff C2_Payoff
0          Axel        Jono     1   Cooperate   Cooperate         2         2
1          Axel        Jono     2   Cooperate   Cooperate         2         2
2          Axel        Jono     3   Cooperate   Cooperate         2         2
3          Axel        Jono     4   Cooperate   Cooperate         2         2
4          Axel        Jono     5   Cooperate   Cooperate         2         2
..          ...         ...   ...         ...         ...       ...       ...
745       Jacob      Touhid    46   Cooperate   Cooperate         2         2
746       Jacob      Touhid    47   Cooperate   Cooperate         2         2
747       Jacob      Touhid    48   Cooperate      Defect         0         3
748       Jacob      Touhid    49   Cooperate      Defect         0         3
749       Jacob      Touhid    50   Cooperate      Defect         0         3

[750 rows x 7 columns]

Score Matrix:
       A